# Geospatio-temporal analysis 

In [58]:
import numpy as np
import pandas as pd
import boto3
import os
from boto3.dynamodb.conditions import Attr
from datetime import datetime

# Export env variables

In [11]:
with open("./app.env") as env_file:
    for env in env_file.read().splitlines():
        key, value = env.split('=')
        os.environ[key] = value

## Download data

In [65]:
dynamodb = boto3.resource('dynamodb', region_name='us-west-1', aws_access_key_id=os.environ['ACCESS_KEY'], aws_secret_access_key=os.environ['SECRET'])
table = dynamodb.Table('luftdaten')
response = table.scan(FilterExpression=Attr('timestamp').contains('2019-03-06 01:53:58'),
                      ProjectionExpression='sensordatavalues, #t, sensor.id, #l.longitude, #l.latitude, #l.altitude',
                      ExpressionAttributeNames = {'#t': 'timestamp', '#l': 'location'},
                      Limit=100)
response_items = response['Items'] 

In [66]:
response_items

[{'location': {'altitude': '347.7',
   'latitude': '48.5840',
   'longitude': '9.3700'},
  'sensor': {'id': Decimal('11465')},
  'sensordatavalues': [{'value': '1.65',
    'value_type': 'P1',
    'id': Decimal('6445939045')},
   {'value': '1.55', 'value_type': 'P2', 'id': Decimal('6445939046')}],
  'timestamp': '2019-03-06 01:53:58'}]

In [67]:
pd_items = []
for item in response_items:
    base_dict = {'latitude': float(item['location']['latitude']),
                 'longitude': float(item['location']['longitude']),
                 'altitude': float(item['location']['altitude']), 
                 **item['sensor'], 
                 'timestamp': datetime.strptime(item['timestamp'], "%Y-%m-%d %H:%M:%S")}
    for sensorvalue in item['sensordatavalues']:
        base_dict[sensorvalue['value_type']] = float(sensorvalue['value'])
    pd_items.append(base_dict)

In [75]:
base_df = pd.DataFrame(pd_items)

In [76]:
base_df

,P1,P2,altitude,id,latitude,longitude,timestamp
0,1.65,1.55,347.7,11465,48.584,9.37,2019-03-06 01:53:58
